Ravy LEON FOUN LIN - Thomas LAHAIE

# Jupyter-Notebook des figures

In [ ]:
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import toml 
import numpy
import matplotlib
import plotly.express as px

Definition du path 

In [ ]:
path = "../../db/metagenomics.sqlite"

## Widgets

In [ ]:
import ipywidgets as widgets
from ipywidgets import Layout

district = widgets.SelectMultiple(
    options = ['porter','ekeley'],
    value = ['porter', 'ekeley'],
    description='Building',
    disabled=False,
    layout = Layout(width='50%', height='80px', display='flex')
)

In [ ]:
district_treemap = widgets.SelectMultiple(
    options = ['porter','ekeley'],
    value = ['porter', 'ekeley'],
    description='Building',
    disabled=False,
    layout = Layout(width='50%', height='80px', display='flex')
)

In [ ]:
district_treemap = widgets.SelectMultiple(
    options = ['porter','ekeley'],
    value = ['porter', 'ekeley'],
    description='Building',
    disabled=False,
    layout = Layout(width='50%', height='80px', display='flex')
)

In [ ]:
place_treemap = widgets.Dropdown(
    options=[('door out', 'door out'), ('door in', 'door in'), ('faucet handles', 'faucet handles'),('stall out','stall out'),
            ('soap dispenser','soap dispenser'),('sink floor','sink floor')],
    value='door out',
    description='Place:',
)

In [ ]:
plot_tr_sb = widgets.Dropdown(
    options=[('sunburn', 'sunburn'), ('treemap', 'treemap')],
    value='sunburn',
    description='plot:',
)

In [ ]:
color_pca = widgets.Dropdown(
    options=[('gender', 'gender'), ('SAMPLE_ID', 'SAMPLE_ID'), ('building', 'buidling'),('surface','surface'),
            ('floor','floor')],
    value='gender',
    description='color_pca:',
)

### Premier barplot

In [ ]:
def premier_plot(path):
         con = sqlite3.connect(path)
         cur = con.cursor()
         command = f"""
         SELECT c.abs_count, s.building, s.surface
         FROM Samples s NATURAL JOIN Counts c
         """
         res = cur.execute(command)
         result = res.fetchall()
         con.commit()
         con.close()


         dict_city = {}
         for item in result :
                  name = item[1]
                  if name not in dict_city.keys():
                           dict_city[name] = float(item[0])
                  else :
                           dict_city[name] += float(item[0])

         df = pd.DataFrame({'Proportion':list(dict_city.values()),'Building_prelevement':list(dict_city.keys())})
         
         fig = px.bar(df,x='Building_prelevement',y='Proportion',title="Proportion d'individus en general dans chaque batiment.",color='Proportion',color_continuous_scale='rdylbu')

         return fig

In [ ]:
premier_plot(path)

### Second barplot

In [ ]:
def update_plot(district):
    dict_fin={}
    
    
    if len(district) == 2 :
        selection_building = district

    else :
        selection_building = (district[0],district[0])
            
    con = sqlite3.connect(path)
    cur = con.cursor()
    command = f"""
        SELECT c.abs_count, s.building, s.surface
        FROM Samples s NATURAL JOIN Counts c
        WHERE s.building = '{selection_building[0]}' OR s.building = '{selection_building[1]}'
    """
    res = cur.execute(command)
    result = res.fetchall()
    con.commit()
    con.close()
    
    dict_elem = {}
    
    for item in result :
        name = item[1]+'_'+item[2]
        if name not in dict_elem.keys() :
            dict_elem[name] = float(item[0])
        else :
            dict_elem[name] += float(item[0])
    
    dict_fin = dict(sorted(dict_elem.items(), key=lambda x:x[1]))
    
    df_barplot = pd.DataFrame({'lieu':[item for item in dict_fin.keys()] ,
                          'proportion':list(dict_fin.values())})
    
    fig = px.bar(df_barplot, x='lieu', y='proportion',color='proportion', color_continuous_scale='PuRd',
             title="Proportion individus retrouver dans chaque environnement.",labels={"lieu":"Environnement de prélévement","proportion":"Proportion"})
    fig.show()

In [ ]:
widgets.interactive(update_plot, district = district)

### Treemap/Sunburst

In [ ]:
def treemap_pltly(district,place,plot):    
    command1 = f"""
            SELECT t.phylum, t.class, t._order, SUM(c.abs_count) as count
            FROM Taxa t NATURAL JOIN Counts as c NATURAL JOIN Samples s
            WHERE s.building = '{district[0]}' AND s.surface = '{place}'
            group by t.phylum, t.class, t._order
        """

    requete = pd.read_sql(command1,sqlite3.connect(path))
    
    if plot == 'sunburn' :
        fig = px.sunburst(requete.fillna(''),path=['class','_order'],values='count',color='class')
    else :
        fig = px.treemap(requete.fillna(''),path=['class','_order'],values='count',color='class')

    fig.show()


In [ ]:
widgets.interactive(treemap_pltly, district = district_treemap,place = place_treemap, plot = plot_tr_sb)

### ACP

In [ ]:
def acp(color):
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA

    con =  sqlite3.connect(path)

    res = pd.read_sql("""
        SELECT c.rel_count, c.OTU_ID, c.SAMPLE_ID, s.gender, s.building, s.surface, s.floor
        FROM Counts c NATURAL JOIN Samples s
    """,con)

    df = res.pivot(index=['SAMPLE_ID','gender','building','surface', "floor"],values='rel_count',columns='OTU_ID')

    SS = StandardScaler()
    SS.fit(df)
    Xnorm = SS.transform(df)

    
    pca = PCA(n_components=2)
    pca.fit(df)

    X_pca = pca.transform(df)
    
    title ="ACP sur le jeu de données avec coloration par genre."
    f=px.scatter(X_pca[:,0],X_pca[:,1],color= df.index.get_level_values(color),title=title)
    f.update_layout( title_x=0.5)

    f.show()


In [ ]:
widgets.interactive(acp, color = color_pca)

### Heatmap

In [ ]:
def heatmap(path) :
    from scipy.spatial.distance import euclidean
    con =  sqlite3.connect(path)

    res3 =pd.read_sql("""
    select *
    from Samples 
    """, con)


    count = pd.read_sql("""
    select rel_count, SAMPLE_ID, OTU_ID from Counts
    """, con)
    count = count.pivot(columns="SAMPLE_ID", index="OTU_ID", values = "rel_count")

    q=res3["SAMPLE_ID"].to_frame()
    qq=q.merge(q, how='cross')
    qq["distance"] = None

    for i, v in qq.iterrows():
        s1, s2 = v["SAMPLE_ID_x"], v["SAMPLE_ID_y"]
        qq.loc[i, "distance"] = euclidean(count[s1],count[s2])


    heatmap=qq.pivot(columns=["SAMPLE_ID_y"],index="SAMPLE_ID_x",values='distance')

    return px.imshow(heatmap,color_continuous_scale='inferno',title="Distance euclidienne de l'abondance relative entre échantillons.")

In [ ]:
heatmap(path)

### Custom barplot

#### data loading

In [ ]:
# data loading and create df

from sqlite3 import connect
from pandas import read_sql


def data_loader(db_path):
    with connect(db_path) as conn: 
        taxa = read_sql("select * from Taxa", conn)
        samples = read_sql("select * from Samples", conn)
        melted = read_sql("select * from Counts", conn)

    melted = melted.set_index(["OTU_ID", "SAMPLE_ID"])
    samples = samples.set_index("SAMPLE_ID")
    taxa = taxa.fillna("").set_index("OTU_ID")

    return melted, samples, taxa


melted, samples, taxa = data_loader(path)

In [ ]:
melted.head()

In [ ]:
taxa.head()

In [ ]:
samples.head()

In [ ]:
#fun batplot custom

def barplot_custom(phylo_choice, sample_choice):
    corr_depth = melted \
        .join(samples, on="SAMPLE_ID") \
        .get(["abs_count"]+[sample_choice]) \
        .groupby(sample_choice) \
        .sum() \
        .rename(columns={"abs_count": "corr_depth"})

    selected_count = melted \
        .join(samples.get(sample_choice), on="SAMPLE_ID") \
        .drop( ["rel_count"], axis="columns") \
        .join(taxa.fillna("").get(phylo_choice), on="OTU_ID") \
        .groupby([phylo_choice] + [sample_choice]) \
        .sum() \
        .join(corr_depth, on=sample_choice) \
        .query("abs_count != 0") \
        .reset_index()

    selected_count["rel_count"] = selected_count["abs_count"] / selected_count["corr_depth"]

    fig = px.bar(selected_count, x=sample_choice, y="rel_count", barmode="stack", color=phylo_choice) 
    
    fig.show()

#### custom bar plot with ipywidget

In [ ]:
sample_choice_widget = widgets.RadioButtons(
    options=('surface', 'gender', 'floor', 'building'), 
    description="choix du type d'echantillon"
)

phylo_choice_widget = widgets.RadioButtons(
    options=('phylum', 'class', '_order', 'family', 'genus', 'specie'),
    description="choix du rang taxonomique"
)

widgets.interactive(barplot_custom, 
    phylo_choice = phylo_choice_widget, 
    sample_choice = sample_choice_widget
)

### Distance Matrix followed by Multidimensional scaling

Instead dnifrac distance matrix, we use euclidean distance matrix (cause Unifrac need Tree with **phylogenetic distance**)

In [ ]:
from sklearn.manifold import MDS

counts = melted \
    .get("rel_count") \
    .to_frame() \
    .reset_index() \
    .pivot(columns="OTU_ID", index="SAMPLE_ID", values="rel_count")

counts.head()

In [ ]:
mds = MDS(dissimilarity="euclidean", n_components=2, normalized_stress='auto')
pos = mds.fit(counts)

pos_df = pd.DataFrame(pos.embedding_, index=counts.index, columns=["x", "y"])
pos_df = pos_df \
    .merge(samples, on="SAMPLE_ID")

pos_df["isFloor"] = pos_df.apply(lambda x: "yes" if "floor" in x["surface"] else "no", axis="columns")

pos_df.head()

In [ ]:
def plot_mds(color_by):
    fig = px.scatter(pos_df, x="x",y="y", 
        color=color_by, width=800,height=700, 
        title="Multi Dimensional Scaling with Euclidean Distance Matrix between Samples"
    )
    fig.show()


choice_color = widgets.RadioButtons(
    options=("gender", "floor", "building", "surface", "isFloor"),
    description="color by"
)

widgets.interactive(plot_mds, 
    color_by = choice_color
)